In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
import albumentations as A
import cv2
import os
import numpy as np
from tqdm import tqdm

C:\Users\dnslab_fox\anaconda3\envs\Ugur\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\dnslab_fox\anaconda3\envs\Ugur\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.7 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
import ultralytics

In [4]:
from ultralytics import YOLO

In [5]:
import torch
import argparse
from torch.utils.data import DataLoader
from ultralytics.utils.torch_utils import select_device
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from ultralytics import solutions
from torchvision import transforms, datasets
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

In [6]:
from ultralytics.nn.modules import *

In [7]:
from yolov5.utils.loss import ComputeLoss

In [7]:
torch.cuda.empty_cache()

In [7]:
import ultralytics
print(ultralytics.__version__)

8.3.101


In [8]:
print(torch.__version__)  # Check PyTorch version
print(torch.version.cuda) # Check CUDA version

2.5.1
11.8


In [5]:
import sys
print(sys.version)

3.12.8 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:48:34) [MSC v.1929 64 bit (AMD64)]


In [9]:
import os
# Rename
# Paths
image_dir = "PearbagsDrone/train/images"
label_dir = "PearbagsDrone/train/labels"

# Get sorted lists
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
label_files = sorted([f for f in os.listdir(label_dir) if f.endswith(".txt")])

# Sanity check
assert len(image_files) == len(label_files), "Number of images and labels must match!"

# Rename label files to match image files
for img_name, lbl_name in zip(image_files, label_files):
    new_lbl_name = os.path.splitext(img_name)[0] + ".txt"
    old_lbl_path = os.path.join(label_dir, lbl_name)
    new_lbl_path = os.path.join(label_dir, new_lbl_name)
    os.rename(old_lbl_path, new_lbl_path)

print("✅ Label files renamed to match image files.")

✅ Label files renamed to match image files.


## Images to Video

In [12]:
import cv2
import os
from natsort import natsorted  # to sort files numerically
#Images to video
# Path to your images folder
image_folder = 'PearbagsVideo/train/images'
output_video_path = 'pearbags_video.avi'
fps = 25  # Adjust depending on how fast you want the video to play

# Get list of image files
images = [img for img in os.listdir(image_folder) if img.endswith(('.jpg', '.png'))]
images = natsorted(images)  # Ensures numerical sorting

# Read the first image to get the video frame size
first_image = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = first_image.shape

# Define video codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Write images to the video
for img_name in images:
    img_path = os.path.join(image_folder, img_name)
    frame = cv2.imread(img_path)
    video.write(frame)

video.release()
print(f"Video saved to: {output_video_path}")

Video saved to: pearbags_video.avi


In [13]:
import cv2
import os

output_path = "pearbags_drone_video.mp4"
fps = 25  # or your original frame rate
width, height = 640, 480  # use actual frame size

# Define the video writer (use 'mp4v' codec for mp4 files)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Example: writing frames
for img_path in sorted(os.listdir("PearbagsDrone/train/images")):  # assume 'frames' folder
    frame = cv2.imread(os.path.join("PearbagsDrone/train/images", img_path))
    resized = cv2.resize(frame, (width, height))
    out.write(resized)

out.release()

## YOLO video and Tracking and Deduplication

In [16]:
!pip install filterpy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110541 sha256=06998abce6051800a6e5791845ef6e5017aac3953a30807bb8c8c0a92f88ae0d
  Stored in directory: c:\users\dnslab_fox\appdata\local\pip\cache\wheels\77\bf\4c\b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy


In [11]:
!pip install deep-sort-realtime

   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ------- -------------------------------- 1.6/8.4 MB 8.4 MB/s eta 0:00:01
   ---------------- ----------------------- 3.4/8.4 MB 8.4 MB/s eta 0:00:01
   ----------------------- ---------------- 5.0/8.4 MB 8.2 MB/s eta 0:00:01
   -------------------------------- ------- 6.8/8.4 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------  8.4/8.4 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 8.4/8.4 MB 7.9 MB/s eta 0:00:00


In [12]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [11]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

Using device: cuda
CUDA Available: True
Number of GPUs: 1


In [10]:
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [14]:
# --- Deduplication using seen track IDs ---
seen_ids = set()

# Paths
video_path = "YoloVideosOutputandInput/pearbags_video.mp4"
output_path = "output_tracked2_deduplicated2.mp4"
model_path = "PearBagsYolo/models/pearbags_cbam.pt"

# Load YOLO model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO(model_path)
model.to(device)

# Load video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, max_iou_distance=0.7)

frame_num = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_num += 1

    # Inference
    results = model.predict(source=frame, device=device, verbose=False)
    detections = []

    for box in results[0].boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box

        # Confidence threshold
        if conf < 0.6:
            continue

        # Class filtering (assuming pearbag class is 0)
        if int(cls) != 0:
            continue

        # Size and aspect ratio filtering
        w_box, h_box = x2 - x1, y2 - y1
        area = w_box * h_box
        aspect_ratio = w_box / h_box if h_box != 0 else 0

        if area < 300 or aspect_ratio > 2.5 or aspect_ratio < 0.3:
            continue

        detections.append(([x1, y1, w_box, h_box], conf, int(cls)))

    # DeepSORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        if track_id not in seen_ids:
            seen_ids.add(track_id)

        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 255, 0), 2)

    cv2.putText(frame, f'Total Unique Pearbags: {len(seen_ids)}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 3)
    out.write(frame)
    print(f"Processed frame {frame_num} | Unique pearbags: {len(seen_ids)}")
# Release resources
cap.release()
out.release()
print("✅ Done! Unique Pearbags Count:", len(seen_ids))
print("🎥 Output saved to:", output_path)

Processed frame 1 | Unique pearbags: 0
Processed frame 2 | Unique pearbags: 0
Processed frame 3 | Unique pearbags: 3
Processed frame 4 | Unique pearbags: 3
Processed frame 5 | Unique pearbags: 3
Processed frame 6 | Unique pearbags: 3
Processed frame 7 | Unique pearbags: 3
Processed frame 8 | Unique pearbags: 3
Processed frame 9 | Unique pearbags: 3
Processed frame 10 | Unique pearbags: 3
Processed frame 11 | Unique pearbags: 3
Processed frame 12 | Unique pearbags: 3
Processed frame 13 | Unique pearbags: 3
Processed frame 14 | Unique pearbags: 3
Processed frame 15 | Unique pearbags: 3
Processed frame 16 | Unique pearbags: 3
Processed frame 17 | Unique pearbags: 3
Processed frame 18 | Unique pearbags: 3
Processed frame 19 | Unique pearbags: 4
Processed frame 20 | Unique pearbags: 4
Processed frame 21 | Unique pearbags: 4
Processed frame 22 | Unique pearbags: 4
Processed frame 23 | Unique pearbags: 4
Processed frame 24 | Unique pearbags: 4
Processed frame 25 | Unique pearbags: 4
Processed

In [14]:
# --- Deduplication using seen track IDs ---
seen_ids = set()

# Paths
video_path = "pearbags_drone_video.mp4"
output_path = "output_tracked_drone_deduplicated.mp4"
model_path = "PearBagsYolo/models/pearbags_cbam.pt"

# Load YOLO model
model = YOLO(model_path)
model.to(device)

# Load video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, max_iou_distance=0.7)

frame_num = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_num += 1

    # Inference
    results = model.predict(source=frame, device=device, verbose=False)
    detections = []
    for box in results[0].boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        if int(cls) == 0:  # Assuming pearbag class is 0
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, int(cls)))

    # DeepSORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        if track_id not in seen_ids:
            seen_ids.add(track_id)

        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Show count on top
    cv2.putText(frame, f'Total Unique Pearbags: {len(seen_ids)}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 3)

    out.write(frame)
    print(f"Processed frame {frame_num} | Unique pearbags: {len(seen_ids)}")

# Cleanup
cap.release()
out.release()
print("✅ Done! Unique Pearbags Count:", len(seen_ids))
print("🎥 Output saved to:", output_path)

Processed frame 1 | Unique pearbags: 0
Processed frame 2 | Unique pearbags: 0
Processed frame 3 | Unique pearbags: 0
Processed frame 4 | Unique pearbags: 0
Processed frame 5 | Unique pearbags: 0
Processed frame 6 | Unique pearbags: 0
Processed frame 7 | Unique pearbags: 0
Processed frame 8 | Unique pearbags: 0
Processed frame 9 | Unique pearbags: 0
Processed frame 10 | Unique pearbags: 0
Processed frame 11 | Unique pearbags: 0
Processed frame 12 | Unique pearbags: 0
Processed frame 13 | Unique pearbags: 0
Processed frame 14 | Unique pearbags: 0
Processed frame 15 | Unique pearbags: 0
Processed frame 16 | Unique pearbags: 0
Processed frame 17 | Unique pearbags: 0
✅ Done! Unique Pearbags Count: 0
🎥 Output saved to: output_tracked_drone_deduplicated.mp4


In [13]:
# Load YOLOv8-CBAM model
model = YOLO("PearBagsYolo/models/pearbags_cbam.pt")  # ← your trained weights

# Initialize DeepSORT
tracker = DeepSort(max_age=30)

# Track unique pearbag IDs
unique_ids = set()

# Load video
cap = cv2.VideoCapture("pearbags_video.mp4")  # ← your input video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Create output video writer
out = cv2.VideoWriter("tracked_output2.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8-CBAM detection
    results = model(frame)[0]
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        conf = float(box.conf)
        cls_id = int(box.cls)
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls_id))

    # DeepSORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = track.to_ltrb()
        unique_ids.add(track_id)

        # Draw box and ID
        cv2.rectangle(frame, (int(l), int(t)), (int(r), int(b)), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Draw counter
    cv2.putText(frame, f"Total Pearbags: {len(unique_ids)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Write and show
    out.write(frame)
    cv2.imshow("Detection + Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"\n✅ Total Unique Pearbags Counted: {len(unique_ids)}")


✅ Total Unique Pearbags Counted: 0


## YOLO-CBAM

In [10]:
model = YOLO("PearBagsYolo/ultralytics/ultralytics/models/yolov8-bifpn-cbam.yaml", task='detect')
model.info()

Parsing layer: 0 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [3, 64, 3, 2]
Parsing layer: 1 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [64, 128, 3, 2]
Parsing layer: 2 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.block.C2f'> | args: [128, 128, 3, True]
Parsing layer: 3 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [128, 256, 3, 2]
Parsing layer: 4 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.block.C2f'> | args: [256, 256, 6, True]
Parsing layer: 5 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [256, 512, 3, 2]
Parsing layer: 6 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.block.C2f'> | args: [512, 512, 6, True]
Parsing layer: 7 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [512, 1024, 3, 2]
Parsing layer: 8 | from: -1 | number: 1 | module: 

(217, 127221368, 127221352, 422.22277280000003)

In [12]:
!yolo task=detect mode=train model=PearBagsYolo/ultralytics/ultralytics/models/yolov8n-bifpn-cbam.yaml data=C:\Users\dnslab_fox\Ugur\PearBagsYolo\data.yaml epochs=50

Parsing layer: 0 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [3, 8, 8, 3, 2]
Parsing layer: 1 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [8, 8, 16, 3, 2]
Parsing layer: 2 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.mycbam.myCBAM'> | args: [8, 16, 16]
Parsing layer: 3 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [16, 16, 32, 3, 2]
Parsing layer: 4 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.mycbam.myCBAM'> | args: [16, 32, 32]
Parsing layer: 5 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [32, 32, 64, 3, 2]
Parsing layer: 6 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.mycbam.myCBAM'> | args: [32, 64, 64]
Parsing layer: 7 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [64, 64, 128, 3, 2]
Parsing layer: 8 | from: -1 | number: 1 | module: <

C:\Users\dnslab_fox\anaconda3\envs\Ugur\Lib\site-packages\torch\nn\init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Traceback (most recent call last):
  File "C:\Users\dnslab_fox\Ugur\PearBagsYolo\ultralytics\ultralytics\engine\trainer.py", line 582, in get_dataset
    data = check_det_dataset(self.args.data)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dnslab_fox\Ugur\PearBagsYolo\ultralytics\ultralytics\data\utils.py", line 372, in check_det_dataset
    raise FileNotFoundError(m)
FileNotFoundError: 
Dataset 'C://Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml' images not found \u26a0\ufe0f, missing path 'C:\Users\dnslab_fox\Ugur\PearBagsYolo\PearBagsYolo\valid\images'
Note dataset download directory is 'C:\Users\dnslab_fox\Ugur\PearBagsYolo'. You can update this in 'C:\Users\dnslab_fox\AppData\Roaming\Ultralytics\settings.json'

The above exception was the direct cause of the following

In [10]:
#model.train(data="PearBagsYolo/data.yaml", epochs=50, imgsz=640)
model.train(data="C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml", epochs=50, imgsz=640)
#model.train(data="path/to/defect.yaml", epochs=50, imgsz=640, batch=16)

New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine\trainer: task=detect, mode=train, model=PearBagsYolo/ultralytics/ultralytics/models/yolov8n-bifpn-cbam.yaml, data=C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agn

train: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\train\labels.cache... 3907 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3907/3907 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\valid\labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to runs\detect\train17\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 97 weight(decay=0.0), 116 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train17
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        500       2176      0.962      0.979      0.989      0.762



50 epochs completed in 46.757 hours.
Optimizer stripped from runs\detect\train17\weights\last.pt, 153.8MB
Optimizer stripped from runs\detect\train17\weights\best.pt, 153.8MB

Validating runs\detect\train17\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
YOLOv8n-bifpn-cbam summary (fused): 144 layers, 76,681,691 parameters, 0 gradients, 191.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:10<00:00,  1.46it/s]


                   all        500       2176      0.966      0.975       0.99      0.763
Speed: 0.3ms preprocess, 16.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train17


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000238741B7A40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [8]:
img = "pearbags/Augmented/20241208_161011.jpg"

In [10]:
model = YOLO("PearBagsYolo/models/pearbags_cbam.pt")

In [11]:
# Inference on an image
results = model(img, save=True, conf=0.25)

# If you want to access results programmatically:
for result in results:
    boxes = result.boxes.xyxy  # x1, y1, x2, y2
    confidences = result.boxes.conf
    class_ids = result.boxes.cls

    print("Detected classes:", class_ids.tolist())
    print("Confidences:", confidences.tolist())
    print("Boxes:", boxes.tolist())


image 1/1 C:\Users\dnslab_fox\Ugur\pearbags\Augmented\20241208_161011.jpg: 480x640 (no detections), 58.5ms
Speed: 3.2ms preprocess, 58.5ms inference, 13.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict4
Detected classes: []
Confidences: []
Boxes: []


In [15]:
model.save("PearBagsYolo/models/pearbags_cbam.pt")

In [32]:
from ultralytics import YOLO
import cv2

# Load trained model
model = YOLO("PearBagsYolo/models/pearbags_cbam.pt")  # Adjust path if needed

# Run inference
results = model(img, conf=0.25, save=False)

# Get result image with boxes
annotated_frame = results[0].plot()  # This adds bounding boxes to the image

# Show the image in a window
annotated_frame = cv2.resize(annotated_frame, (640, 640))
cv2.imshow("Detection Result", annotated_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\dnslab_fox\Ugur\pearbags\Augmented\20241208_161011.jpg: 480x640 (no detections), 18.6ms
Speed: 2.1ms preprocess, 18.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)


In [33]:
results = model(img)
count = len(results[0].boxes)
print(f"Detected pearbags: {count}")


image 1/1 C:\Users\dnslab_fox\Ugur\pearbags\Augmented\20241208_161011.jpg: 480x640 (no detections), 244.6ms
Speed: 4.4ms preprocess, 244.6ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Detected pearbags: 0


In [24]:
from ultralytics import YOLO
import cv2

# Load trained model
model = YOLO("PearBagsYolo/models/pearbags_cbam.pt")  # use your .pt file path

# Run inference on the image
results = model(img)  # change to your image path

# Get number of detected objects
count = len(results[0].boxes)

# Plot image with boxes (Ultralytics draws it for you)
annotated_img = results[0].plot()

# Add count as text on top-left corner
cv2.putText(
    annotated_img,
    f"Detected pearbags: {count}",
    (10, 30),  # x, y position
    cv2.FONT_HERSHEY_SIMPLEX,
    1,              # font scale
    (0, 255, 0),    # color (green)
    2,              # thickness
    cv2.LINE_AA
)

# Show the image in notebook or pop-up window
cv2.imshow("Detection with Count", annotated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\dnslab_fox\Ugur\PearBagsYolo\valid\images\PE_001_TP_20231110_PM_001_007_001_0559_IMG_jpg.rf.a5c22c47f443a3bcaa8c423125b56356.jpg: 640x640 4 pearbagss, 25.9ms
Speed: 2.1ms preprocess, 25.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


## YOLO-CBAM-BiFPN

In [10]:
model_cbam_bifpn = YOLO("PearBagsYolo/ultralytics/ultralytics/models/yolov8-bifpn-cbam.yaml", task='detect')
model_cbam_bifpn.info()

Parsing layer: 0 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [3, 64, 3, 2]
Parsing layer: 1 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [64, 128, 3, 2]
Parsing layer: 2 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.block.C2f'> | args: [128, 128, 3, True]
Parsing layer: 3 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [128, 256, 3, 2]
Parsing layer: 4 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.block.C2f'> | args: [256, 256, 6, True]
Parsing layer: 5 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [256, 512, 3, 2]
Parsing layer: 6 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.block.C2f'> | args: [512, 512, 6, True]
Parsing layer: 7 | from: -1 | number: 1 | module: <class 'ultralytics.nn.modules.conv.Conv'> | args: [512, 1024, 3, 2]
Parsing layer: 8 | from: -1 | number: 1 | module: 

(217, 127221368, 127221352, 422.22277280000003)

In [11]:
#model.train(data="PearBagsYolo/data.yaml", epochs=50, imgsz=640)
model_cbam_bifpn.train(data="C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml", epochs=50, imgsz=640)
#model.train(data="path/to/defect.yaml", epochs=50, imgsz=640, batch=16)

New https://pypi.org/project/ultralytics/8.3.111 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine\trainer: task=detect, mode=train, model=PearBagsYolo/ultralytics/ultralytics/models/yolov8-bifpn-cbam.yaml, data=C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train19, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agno

train: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\train\labels.cache... 3907 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3907/3907 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\valid\labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to runs\detect\train19\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 97 weight(decay=0.0), 108 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train19
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        500       2176      0.955      0.976      0.986       0.76



50 epochs completed in 151.062 hours.
Optimizer stripped from runs\detect\train19\weights\last.pt, 254.9MB
Optimizer stripped from runs\detect\train19\weights\best.pt, 254.9MB

Validating runs\detect\train19\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
YOLOv8-bifpn-cbam summary (fused): 120 layers, 127,187,896 parameters, 0 gradients, 421.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:16<00:00,  1.06s/it]


                   all        500       2176      0.955      0.976      0.986       0.76
Speed: 0.4ms preprocess, 28.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train19


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020281010F20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [12]:
model_cbam_bifpn.save("PearBagsYolo/models/pearbags_cbam_bifpn.pt")

In [15]:
img = "PearBagsYolo/train/images/PE_001_TP_20231110_PM_001_007_000_0013_IMG_jpg.rf.a84906c32b25aa5e6cf54f5783511d09.jpg"

In [16]:
# Inference on an image
results = model_cbam_bifpn(img, save=True, conf=0.25)

# If you want to access results programmatically:
for result in results:
    boxes = result.boxes.xyxy  # x1, y1, x2, y2
    confidences = result.boxes.conf
    class_ids = result.boxes.cls

    print("Detected classes:", class_ids.tolist())
    print("Confidences:", confidences.tolist())
    print("Boxes:", boxes.tolist())


image 1/1 C:\Users\dnslab_fox\Ugur\PearBagsYolo\train\images\PE_001_TP_20231110_PM_001_007_000_0013_IMG_jpg.rf.a84906c32b25aa5e6cf54f5783511d09.jpg: 640x640 6 pearbagss, 370.7ms
Speed: 4.1ms preprocess, 370.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\train193
Detected classes: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Confidences: [0.8678127527236938, 0.8645520210266113, 0.8521826267242432, 0.8391143679618835, 0.8329914808273315, 0.8201019763946533]
Boxes: [[24.674413681030273, 185.97958374023438, 52.296043395996094, 238.98593139648438], [142.97705078125, 98.24432373046875, 172.49835205078125, 139.27664184570312], [530.2200317382812, 212.7557830810547, 547.6135864257812, 250.8553009033203], [278.29156494140625, 213.01895141601562, 289.6064453125, 233.24728393554688], [222.752197265625, 208.2711181640625, 237.81304931640625, 232.73480224609375], [454.77423095703125, 219.04605102539062, 467.82965087890625, 243.36288452148438]]


In [17]:
from ultralytics import YOLO
import cv2

# Load trained model
model_cbam_bifp = YOLO("PearBagsYolo/models/pearbags_cbam_bifpn.pt")  # Adjust path if needed

# Run inference
results = model_cbam_bifp(img, conf=0.25, save=False)

# Get result image with boxes
annotated_frame = results[0].plot()  # This adds bounding boxes to the image

# Show the image in a window
annotated_frame = cv2.resize(annotated_frame, (640, 640))
cv2.imshow("Detection Result", annotated_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\dnslab_fox\Ugur\PearBagsYolo\train\images\PE_001_TP_20231110_PM_001_007_000_0013_IMG_jpg.rf.a84906c32b25aa5e6cf54f5783511d09.jpg: 640x640 6 pearbagss, 84.9ms
Speed: 2.8ms preprocess, 84.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


## YOLOv11

In [9]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml", epochs=50, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.116 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train21, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

train: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\train\labels.cache... 3907 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3907/3907 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\valid\labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to runs\detect\train21\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train21
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.67G     0.9803     0.4229     0.8814         13        640: 100%|██████████| 245/245 [00:25<00:00,  9.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02<00:00,  6.54it/s]

                   all        500       2176      0.963       0.98      0.988      0.737



50 epochs completed in 0.415 hours.
Optimizer stripped from runs\detect\train21\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train21\weights\best.pt, 5.5MB

Validating runs\detect\train21\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03<00:00,  4.55it/s]


                   all        500       2176      0.968      0.981      0.989      0.739
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train21


In [10]:
model.save("PearBagsYolo/models/pearbags_yolov11.pt")

## Yolov8

In [11]:
# Load a COCO-pretrained YOLOv8n model
model_v8 = YOLO("yolov8n.pt")

# Display model information (optional)
model_v8.info()

# Train the model on the COCO8 example dataset for 100 epochs
model_v8.train(data="C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml", epochs=50, imgsz=640)

100%|██████████| 6.25M/6.25M [00:00<00:00, 11.6MB/s]


YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
New https://pypi.org/project/ultralytics/8.3.116 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/dnslab_fox/Ugur/PearBagsYolo/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, 

train: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\train\labels.cache... 3907 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3907/3907 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\dnslab_fox\Ugur\PearBagsYolo\valid\labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


...

                   all        500       2176      0.968      0.983      0.991      0.749



50 epochs completed in 0.389 hours.
Optimizer stripped from runs\detect\train22\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train22\weights\best.pt, 6.2MB

Validating runs\detect\train22\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03<00:00,  4.74it/s]


                   all        500       2176      0.968      0.983      0.991      0.749
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train22


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001A56AD56090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [12]:
model_v8.save("PearBagsYolo/models/pearbags_yolov8.pt")

## Change Label

In [10]:
import os

labels_dir = "PearBagsYolo/train/labels"
bad_files = []

for file in os.listdir(labels_dir):
    if file.endswith(".txt"):
        path = os.path.join(labels_dir, file)
        with open(path, "r") as f:
            lines = f.readlines()
            if not lines:
                bad_files.append(file)
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5 or not all(p.replace('.', '', 1).isdigit() for p in parts[1:]):
                    bad_files.append(file)
                    break

print("Bad label files:", bad_files)

Bad label files: []


In [14]:
import os

labels_dir = "PearBagsYolo/test/labels"  # repeat for valid/test if needed

for filename in os.listdir(labels_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(labels_dir, filename)
        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts[0] == "1":
                parts[0] = "0"
            new_lines.append(" ".join(parts))

        with open(file_path, "w") as f:
            f.write("\n".join(new_lines))